## Setup

### Imports

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import math
import time
from datetime import timedelta
import matplotlib.pyplot as plt
from decimal import Decimal, ROUND_HALF_UP
from typing import List

import pulp
import cvxpy as cp
import gurobipy as gp

from sklearn.model_selection import train_test_split
from scipy.stats import pearsonr
from scipy.stats import spearmanr

import data_preparation
from data import ppmi_data_loader
from evaluation import Evaluator

### Create encoded df

In [ ]:
def is_self_reported(item):
    for prefix in ["NP1SLPN", "NP1SLPD", "NP1PAIN", "NP1URIN", "NP1CNST", "NP1LTHD", "NP1FATG", "NP2"]:
        if item.startswith(prefix):
            return True
    return False

df, items, all_items = data_preparation.get_encoded_df(include_moca=True)      
self_reported_items = [item for item in all_items if is_self_reported(item)]

### Train test split

In [ ]:
all_patnos = df.PATNO.unique().tolist()

train_patients, test_patients = train_test_split(all_patnos, test_size=0.2, random_state=42)

train_df = df[df['PATNO'].isin(train_patients)]
test_df = df[df['PATNO'].isin(test_patients)]

### Summary statistics

In [ ]:
print(f"Total number of items: {len(all_items)}")
print(f"\nNumber of self reported items: {len(self_reported_items)}")
print(f"\nTotal number of visits: {len(df)}")
print(f"\nTotal number of unique patients: {len(df.PATNO.unique())}")

In [ ]:
# H&Y distribution
print(f"Ratio of visits with H&Y <=2: {len(df[df.NHY <= 2]) / len(df)}")

nhy_counts = pd.DataFrame({
    'Train': train_df.NHY.value_counts(dropna=False).sort_index(),
    'Test': test_df.NHY.value_counts(dropna=False).sort_index()
})

display(nhy_counts.T)

## Optimize

In [ ]:
from optimizers.naive_p1_optimizer import NaiveP1Optimizer
from optimizers.naive_p2_optimizer import NaiveP2Optimizer
from optimizers.naive_p3_optimizer import NaiveP3Optimizer
from optimizers.naive_mds_updrs_optimizer import NaiveMdsUpdrsOptimizer
from optimizers.naive_moca_optimizer import NaiveMocaOptimizer
from optimizers.linear_programming_optimizer import LinearProgrammingOptimizer
from optimizers.weighted_linear_programming_optimizer import WeightedLinearProgrammingOptimizer
from optimizers.quadratic_programming_optimizer import QuadraticProgrammingOptimizer
from optimizers.mean_variance_balance_optimizer import MeanVarianceBalanceOptimizer
from optimizers.mixed_integer_programming_optimizer import MixedIntegerProgrammingOptimizer
from optimizers.integer_programming_optimizer import IntegerProgrammingOptimizer
from optimizers.weights_optimizer import Status

TIME_LIMIT = 86400  # 24 hours

def create_optimizers(data, items, version):
    """Creates a list of optimizer instances."""
    return [
        NaiveP1Optimizer(data=data, items=items, version=version),
        NaiveP2Optimizer(data=data, items=items, version=version),
        NaiveP3Optimizer(data=data, items=items, version=version),
        NaiveMdsUpdrsOptimizer(data=data, items=items, version=version),
        NaiveMocaOptimizer(data=data, items=items, version=version),
        LinearProgrammingOptimizer(data=data, items=items, version=version, name='MeanDiff'),
        WeightedLinearProgrammingOptimizer(data=data, items=items, version=version, name='MeanDiff-W'),
        QuadraticProgrammingOptimizer(data=data, items=items, version=version, name='MeanDiff-QP'),
        MeanVarianceBalanceOptimizer(data=data, items=items, version=version, name='MeanDiff-SV'),
        MixedIntegerProgrammingOptimizer(data=data, items=items, time_limit=TIME_LIMIT, version=version, name='Cons'),
        IntegerProgrammingOptimizer(data=data, items=items, time_limit=TIME_LIMIT, version=version, name='Cons-Int'),
    ]

def run_optimizers(optimizers):
    """Runs the optimization process for a list of optimizers, reusing results if already completed."""
    results = {}
    total_start_time = time.time()

    for i, opt in enumerate(optimizers):
        name = opt.get_name()
        print(f"[{i+1}/{len(optimizers)}] Starting {name}")

        # Check if the optimizer already completed successfully
        status = opt.get_status()
        if status == Status.DONE:
            print(f"{name} already completed. Retrieving stored weights.")
            weights = opt.get_weights()
        else:
            # Run the optimization process
            start_time = time.time()
            weights, status = opt.calc_weights()
            run_time = time.time() - start_time
            print(f"{name} done.\nStatus = {status}.\nRuntime: {str(timedelta(seconds=run_time))}")

        # Save the results if not an error
        if status != Status.ERROR:
            results[name] = weights
        else:
            print(f"Optimizer {name} failed.")

    total_run_time = time.time() - total_start_time
    print(f"Total runtime for all optimizers: {str(timedelta(seconds=total_run_time))}")
    return results

optimizers_all_items = create_optimizers(data=train_df, items=all_items, version="all_items")
results_all_items = run_optimizers(optimizers_all_items)

optimizers_self_report = create_optimizers(data=train_df, items=self_reported_items, version="self_reported")
results_self_report = run_optimizers(optimizers_self_report)

### Save weights

In [ ]:
# Get the current epoch timestamp
timestamp = int(time.time())

# Convert the results_all_items dict to a DataFrame with 'item' as the first column
df_all_items = pd.DataFrame({'item': all_items, **results_all_items})

# Convert the results_self_report dict to a DataFrame with 'item' as the first column
df_self_report = pd.DataFrame({'item': self_reported_items, **results_self_report})

# Save the DataFrames as CSV files with the timestamp in the filename
df_all_items.to_csv(f'optimizers/weights/all_items_weights_{timestamp}.csv', index=False)
df_self_report.to_csv(f'optimizers/weights/self_report_weights_{timestamp}.csv', index=False)

### Load weights (Optional)

To examine weights calculated in a previous run

In [ ]:
# version = '123456789'  # The timestamp when version was saved

# df_all_items = pd.read_csv(f'optimizers/weights/all_items_weights_{version}.csv')
# df_self_report = pd.read_csv(f'optimizers/weights/self_report_weights_{version}.csv')

# results_all_items = {col: df_all_items[col].tolist() for col in df_all_items.columns if col != 'item'}
# results_self_report = {col: df_self_report[col].tolist() for col in df_self_report.columns if col != 'item'}

## Analyze results

### Consistency

In [ ]:
# Create the consistency tables

evaluator_all_items = Evaluator(test_df=test_df, all_items=all_items)
evaluator_self_report = Evaluator(test_df=test_df, all_items=self_reported_items)

num_pairs_all = evaluator_all_items.get_num_pairs()
num_pairs_self_report = evaluator_self_report.get_num_pairs()

# Convert months to years
num_pairs_all.index = num_pairs_all.index // 12
num_pairs_self_report.index = num_pairs_self_report.index // 12

# Ignore rows with more than 10 years gaps (too scarce)
num_pairs_all = num_pairs_all[num_pairs_all.index <= 10]
num_pairs_self_report = num_pairs_self_report[num_pairs_self_report.index <= 10]

data_all_items = {"Number of Pairs": num_pairs_all}
data_self_report = {"Number of Pairs": num_pairs_self_report}

# All Items
total_weighted_avg_all = {}
for name, res in results_all_items.items():
    percentage_scores_pos = evaluator_all_items.get_increase_percentages(res)
    percentage_scores_pos.index = percentage_scores_pos.index // 12
    percentage_scores_pos = percentage_scores_pos[percentage_scores_pos.index <= 10]
    data_all_items[name] = percentage_scores_pos
    total_weighted_avg_all[name] = (percentage_scores_pos * num_pairs_all).sum() / num_pairs_all.sum()

# Self-Reported Items
total_weighted_avg_self = {}
for name, res in results_self_report.items():
    percentage_scores_pos = evaluator_self_report.get_increase_percentages(res)
    percentage_scores_pos.index = percentage_scores_pos.index // 12
    percentage_scores_pos = percentage_scores_pos[percentage_scores_pos.index <= 10]
    data_self_report[name] = percentage_scores_pos
    
    total_weighted_avg_self[name] = (percentage_scores_pos * num_pairs_self_report).sum() / num_pairs_self_report.sum()

inc_all_items_df = pd.DataFrame(data_all_items).sort_index()
inc_self_report_df = pd.DataFrame(data_self_report).sort_index()

inc_all_items_df.index.name = "Years Gap"
inc_self_report_df.index.name = "Years Gap"

weighted_avg_all_df = pd.DataFrame.from_dict(total_weighted_avg_all, orient='index', columns=["Total Weighted Increase (%)"])
weighted_avg_self_df = pd.DataFrame.from_dict(total_weighted_avg_self, orient='index', columns=["Total Weighted Increase (%)"])

all_items_cons_table = inc_all_items_df.copy()
self_reported_items_cons_table = inc_self_report_df.copy()
all_items_cons_table.loc['All'] = [num_pairs_all.sum()] + list(total_weighted_avg_all.values())
self_reported_items_cons_table.loc['All'] = [num_pairs_self_report.sum()] + list(total_weighted_avg_self.values())

In [ ]:
# Display the consistency for each method and time gap for methods using all items
display(all_items_cons_table.round(2).T)

In [ ]:
# Display the consistency for each method and time gap for methods using only self reported items
display(self_reported_items_cons_table.round(2).T)

In [ ]:
# Compare MDS-UPDRS to MeanDiff-QP using all items or only self reported items

fig, ax = plt.subplots(figsize=(10, 6))

blue_colors = sns.color_palette("Blues", 2)
green_colors = sns.color_palette("Oranges", 2)

sns.lineplot(data=inc_all_items_df['MDS-UPDRS'], label=f'MDS-UPDRS (All Items)', ax=ax, linestyle='--', color=blue_colors[0])
sns.lineplot(data=inc_self_report_df['MDS-UPDRS'], label=f'MDS-UPDRS (Self-Reported)', ax=ax, linestyle='--', color=green_colors[0])
sns.lineplot(data=inc_all_items_df['MeanDiff-QP'], label=f'MeanDiff-QP (All Items)', ax=ax, linestyle='-', color=blue_colors[1])
sns.lineplot(data=inc_self_report_df['MeanDiff-QP'], label=f'MeanDiff-QP (Self-Reported)', ax=ax, linestyle='-', color=green_colors[1])

ax.set_xlabel('Time gap (years)')
ax.set_xticks(inc_all_items_df.index)
ax.set_ylabel('Percentage of consistent pairs')
ax.legend()

plt.tight_layout()
plt.show()

### Consistency vs parsity

In [ ]:
def create_pareto_df(weighted_avg_df, results_dict):
    # Extract 'Total Weighted Increase (%)' for each method
    consistency = weighted_avg_df['Total Weighted Increase (%)']

    # Count non-zero weights for each method
    non_zero = {method: sum(1 for w in weights if w != 0) for method, weights in results_dict.items()}

    # Convert to DataFrame
    pareto_df = pd.DataFrame({
        'consistency': consistency,
        'non_zero': pd.Series(non_zero)
    }).reset_index()

    return pareto_df

# Creating Pareto DataFrames
pareto_all_items_df = create_pareto_df(weighted_avg_all_df, results_all_items)
pareto_self_reported_df = create_pareto_df(weighted_avg_self_df, results_self_report)

# Remove methods irrelevant for self report
pareto_self_reported_df = pareto_self_reported_df[pareto_self_reported_df.non_zero > 0]

In [ ]:
# Display the consistency and sparsity of all method using all items
display(pareto_all_items_df)

In [ ]:
# Display the consistency and sparsity of all method using only self reported items
display(pareto_self_reported_df)

In [ ]:
# Draw Pareto frontiers 
unique_methods = sorted(set(pareto_all_items_df['index']).union(set(pareto_self_reported_df['index'])))
markers = dict(zip(unique_methods, ['^', '^', 's', 's', 's', 's', 'p', 'p', 'p', 'p', 'D'][:len(unique_methods)]))
orange_colors = sns.color_palette("Oranges_r", 5)
green_colors = sns.color_palette("summer", 6)

# Assign colors based on markers
palette = {}
for i, method in enumerate(unique_methods):
    if markers[method] in {'s', 'D'}:
        palette[method] = orange_colors.pop(0)  # Assign a deep orange color
    else:
        palette[method] = green_colors.pop(0)  # Assign a strong greenish color


def plot_pareto_frontier(ax, df, title):
    scatter = sns.scatterplot(data=df, x='non_zero', y='consistency', hue='index', style='index', 
                              s=300, ax=ax, palette=palette, markers=markers)

    pareto_front = df.sort_values('non_zero').copy()
    pareto_front['Pareto'] = pareto_front['consistency'].cummax()
    pareto_points = pareto_front[pareto_front['consistency'] == pareto_front['Pareto']]

    ax.plot(pareto_points['non_zero'], pareto_points['consistency'], color='red', linestyle='--', label='Pareto Frontier')

    ax.set_title(title, fontsize=18)
    ax.set_xlabel('Number of Non-zero Items', fontsize=16)
    ax.set_ylabel('Consistent Pairs Percentage', fontsize=16)

    return scatter  # Return scatterplot to extract legend handles and labels

fig, axes = plt.subplots(1, 2, figsize=(18, 8), sharey=True)

scatter = plot_pareto_frontier(axes[0], pareto_all_items_df, 'All Items - Pareto Frontier')
plot_pareto_frontier(axes[1], pareto_self_reported_df.drop(columns=[]), 'Self-Reported Items - Pareto Frontier')

axes[0].get_legend().remove()
axes[1].get_legend().remove()

handles, labels = scatter.get_legend_handles_labels()

# Place a single legend in between the two plots
fig.legend(handles, labels, loc='lower center', fontsize=20, markerscale=2, ncol=3, frameon=True, bbox_to_anchor=(0.5, 0.1))

# Add subplot labels
axes[0].text(0.05, 1.05, 'A', transform=axes[0].transAxes, fontsize=20, fontweight='bold', va='top', ha='right')
axes[1].text(0.05, 1.05, 'B', transform=axes[1].transAxes, fontsize=20, fontweight='bold', va='top', ha='right')

plt.tight_layout()
plt.show()

### Compare to external metrics

In [ ]:
dl = ppmi_data_loader.PpmiDataLoader(ppmi_dir='data/PPMI')

#### Time to levodopa

In [ ]:
def process_dopaminergic_data(dop_df):
    dop_df['INFODT'] = pd.to_datetime(dop_df['INFODT'], format='%m/%Y')
    dop_df = dop_df.sort_values(by=['PATNO', 'INFODT'])
    
    def get_levodopa_start(group):
        if (group['DOPTHERST'] == 0).any():
            first_start = group[group['DOPTHERST'] == 1].head(1)
            return first_start
        return pd.DataFrame()

    levodopa_start_df = dop_df.groupby('PATNO').apply(get_levodopa_start).reset_index(drop=True)
    return levodopa_start_df.groupby('PATNO')['INFODT'].min().to_dict()

def apply_levodopa_dates(test_df_copy, earliest_dates):
    """
    Apply the earliest levodopa start dates to the test data.
    """
    test_df_copy.loc[:, 'levodopa_start_date'] = test_df_copy['PATNO'].map(earliest_dates)
    test_df_copy.loc[:, 'time_diff'] = (test_df_copy['levodopa_start_date'] - test_df_copy['INFODT']).dt.days
    return test_df_copy.dropna(subset=['time_diff'])

def calc_correlations(final_scores_df, df_weights, prefix, methods):
    for i, method in enumerate(methods):
        x = final_scores_df[final_scores_df['method'] == method]['time_diff'].dropna()
        y = final_scores_df[final_scores_df['method'] == method]['total_score'].dropna()
        r, p = pearsonr(x, y)
        print(f'{prefix}, {method}: rho={r}, p={p}')

dop_df = dl.get_dopa_start_df()
earliest_dates = process_dopaminergic_data(dop_df)

weighted_scores_all_items_df = evaluator_all_items.calculate_all_weighted_scores(df_all_items)
weighted_scores_all_items_df = apply_levodopa_dates(weighted_scores_all_items_df, earliest_dates)

weighted_scores_self_report_df = evaluator_self_report.calculate_all_weighted_scores(df_self_report)
weighted_scores_self_report_df = apply_levodopa_dates(weighted_scores_self_report_df, earliest_dates)

all_methods = inc_all_items_df.columns.tolist()[1:]
self_reported_methods = [m for m in all_methods if m not in ['MDS-UPDRS Part 3', 'MoCA']]

calc_correlations(weighted_scores_all_items_df, df_all_items, prefix='All Items', methods=all_methods)
calc_correlations(weighted_scores_self_report_df, df_self_report, prefix='Self-Reported Items', methods=self_reported_methods)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import pearsonr

def plot_selected_methods(final_scores_df, method, title, label, ax):
    """
    Plots a single method's scores vs. years before levodopa treatment on a given axis,
    with a trend line.
    """
    x = final_scores_df[final_scores_df['method'] == method]['time_diff'].dropna() / 365.25
    y = final_scores_df[final_scores_df['method'] == method]['total_score'].dropna()

    r, p = pearsonr(x, y)
    ax.scatter(x, y, label=f'{method} {title}', color='blue')

    # Fit and plot a trend line
    coeffs = np.polyfit(x, y, 1)  # Linear fit
    poly_eq = np.poly1d(coeffs)
    x_fit = np.linspace(min(x), max(x), 100)
    y_fit = poly_eq(x_fit)
    ax.plot(x_fit, y_fit, linestyle='--', color='red', label='Trend Line')

    ax.text(0.05, 0.05, f'Pearson ρ: {r:.2f}\np: {p:.2e}', 
            verticalalignment='bottom', horizontalalignment='left', 
            transform=ax.transAxes, fontsize=12, bbox=dict(facecolor='white', alpha=0.5))

    ax.set_title(f'{method} ({title})\nScores vs. Years Before Treatment', fontsize=18)
    ax.set_xlabel('Years Before Levodopa', fontsize=16)
    ax.set_ylabel(f'{method} Score', fontsize=16)
    ax.grid(True)

    # Add subplot label ('A' or 'B') in top-left corner
    ax.text(0.04, 1.06, label, transform=ax.transAxes, fontsize=20, fontweight='bold', va='top', ha='right')

fig, axes = plt.subplots(1, 2, figsize=(18, 6))  # No 'sharey=True' here

# Plot 'MeanDiff-QP' for all items (left plot, labeled 'A')
plot_selected_methods(weighted_scores_all_items_df, 'MeanDiff-QP', 'All Items', 'A', axes[0])

# Plot 'Cons-Int' for self-reported items (right plot, labeled 'B')
plot_selected_methods(weighted_scores_self_report_df, 'Cons-Int', 'Self-Reported Items', 'B', axes[1])

plt.tight_layout()
plt.show()

#### S&E ADL

In [ ]:
se_adl_df = dl.get_se_adl_df()
se_adl_df['INFODT'] = pd.to_datetime(se_adl_df['INFODT'], format='%m/%Y')

test_df_copy = test_df.copy()
filtered_df = test_df_copy.merge(se_adl_df, on=['PATNO', 'EVENT_ID', 'INFODT'], how='inner')

def calculate_weighted_scores(row, weights_df):
    """
    Calculate the weighted sum for each weighting method for a given row.
    """
    weighted_scores = {}
    for method in weights_df.columns[1:]:
        weights = weights_df.set_index('item')[method]
        weighted_scores[method] = sum(row[item] * weights[item] for item in weights.index if item in row.index)
    return pd.Series(weighted_scores)

def calc_correlations(filtered_df, df_weights, prefix):
    """
    Apply the weighted score calculation and plot for a given set of weights (df_weights).
    """
    # Apply the weighted score calculation for each row
    weighted_scores_df = filtered_df.apply(lambda row: calculate_weighted_scores(row, df_weights), axis=1)

    # Combine the results with SE ADL data
    final_df = pd.concat([filtered_df[['PATNO', 'INFODT', 'MSEADLG']], weighted_scores_df], axis=1)

    for i, method in enumerate(df_weights.columns[1:]):
        x = final_df['MSEADLG']
        y = final_df[method]
        
        data = final_df[['MSEADLG', method]].copy()
        
        if not data.empty and not all(data[method] == data[method].iloc[0]):
            # r, p = spearmanr(data['MSEADLG'], data[method])
            r, p = pearsonr(data['MSEADLG'], data[method])
            print(f'{prefix}, {method}: rho={r}, p={p}')

calc_correlations(filtered_df, df_all_items, 'All Items')
calc_correlations(filtered_df, df_self_report, 'Self-Reported Items')

In [ ]:
# Step 1: Compute Weighted Scores First
def compute_weighted_scores(filtered_df, df_weights):
    """
    Computes the weighted sum for each method in df_weights.
    """
    weighted_scores_df = filtered_df.apply(lambda row: calculate_weighted_scores(row, df_weights), axis=1)
    return pd.concat([filtered_df[['PATNO', 'INFODT', 'MSEADLG']], weighted_scores_df], axis=1)

weighted_adl_all_items_df = compute_weighted_scores(filtered_df, df_all_items)
weighted_adl_self_report_df = compute_weighted_scores(filtered_df, df_self_report)

def plot_selected_adl_methods(weighted_df, method, title, label, ax):
    """
    Plots a single method's scores vs. S&E ADL score on a given axis.
    """
    if method not in weighted_df.columns:
        print(f"Warning: Method '{method}' not found in dataframe columns.")
        ax.text(0.5, 0.5, 'No data available', horizontalalignment='center', 
                verticalalignment='center', transform=ax.transAxes, fontsize=14)
        return

    x = weighted_df['MSEADLG']
    y = weighted_df[method]

    data = weighted_df[['MSEADLG', method]].dropna()

    # Filter groups with at least 5 data points
    filtered_data = data.groupby('MSEADLG').filter(lambda x: len(x) >= 5)

    # Calculate Spearman correlation
    r, p = pearsonr(data['MSEADLG'], data[method])

    # Plot boxplot
    sns.boxplot(x='MSEADLG', y=method, data=filtered_data, ax=ax, color='skyblue')

    # Add correlation text
    ax.text(0.95, 0.05, f'Pearson ρ: {r:.2f}\np: {p:.2e}',
            verticalalignment='bottom', horizontalalignment='right',
            transform=ax.transAxes, fontsize=12, bbox=dict(facecolor='white', alpha=0.5))

    # Title with two lines
    ax.set_title(f'{method} ({title})\nScores vs. S&E ADL Score', fontsize=18)
    ax.set_xlabel('S&E ADL Score', fontsize=16)
    ax.set_ylabel(f'{method} Score', fontsize=16)
    ax.grid(True)

    # Add subplot label ('A' or 'B')
    ax.text(0.04, 1.06, label, transform=ax.transAxes, fontsize=20, fontweight='bold', va='top', ha='right')

fig, axes = plt.subplots(1, 2, figsize=(18, 6))

plot_selected_adl_methods(weighted_adl_all_items_df, 'MeanDiff-QP', 'All Items', 'A', axes[0])
plot_selected_adl_methods(weighted_adl_self_report_df, 'MeanDiff-QP', 'Self-Reported Items', 'B', axes[1])

plt.tight_layout()
plt.show()

#### Time to milestone

In [ ]:
# Add relevant data for milestoen calculation

scopa_aut_df = pd.read_csv('data/PPMI/non_motor_assessments/SCOPA-AUT_07Aug2024.csv')
scopa_aut_df = scopa_aut_df.drop(columns=['REC_ID', 'PAG_NAME', 'INFODT', 'PTCGBOTH','ORIG_ENTRY', 'LAST_UPDATE'])
merged_df = pd.merge(df, scopa_aut_df, on=['PATNO','EVENT_ID'], how='inner')

moca_df = pd.read_csv('data/PPMI/non_motor_assessments/Montreal_Cognitive_Assessment__MoCA__07Aug2024.csv')
moca_df = moca_df[['PATNO', 'EVENT_ID', 'MCATOT']]
merged_df = pd.merge(merged_df, moca_df, on=['PATNO','EVENT_ID'], how='inner')

se_df = pd.read_csv('data/PPMI/motor_assessments/Modified_Schwab___England_Activities_of_Daily_Living_07Aug2024.csv')
se_df = se_df[['PATNO', 'EVENT_ID', 'MSEADLG']]
merged_df = pd.merge(merged_df, se_df, on=['PATNO','EVENT_ID'], how='inner')

In [ ]:
# Add milestones

merged_df['milestone_scopa_aut_syncope'] = (merged_df.SCAU16 >= 1).astype(int)
merged_df['milestone_orthostatic_hypotension'] = (merged_df.SCAU15 >= 2).astype(int)
merged_df['milestone_urinary_incontinence'] = ((merged_df.NP1URIN_th3 == 1) & ((merged_df.SCAU8 >= 2) | (merged_df.SCAU9 >= 2))).astype(int)
merged_df['milestone_moca'] = (merged_df.MCATOT < 21).astype(int)
merged_df['milestone_h_and_y'] = (merged_df.NHY >= 4).astype(int)
merged_df['milestone_s_and_e'] = (merged_df.MSEADLG < 80).astype(int)

milestone_cols = ["NP2WALK_th3",
                  "NP2FREZ_th3",
                  "NP3GAIT_th3",
                  "NP3FRZGT_th4",
                  "NP3PSTBL_th3",
                  "NP1COG_th3",
                  "NP1HALL_th3",
                  "NP1APAT_th3",
                  "NP1LTHD_th4",
                  "NP2SWAL_th3",
                  "NP2EAT_th3",
                  "NP2DRES_th3",
                  "NP2HYGN_th3",
                  "NP3SPCH_th3",
                 ]

additional_milestones = ["milestone_moca",
                         "milestone_scopa_aut_syncope",
                         "milestone_orthostatic_hypotension",
                         "milestone_urinary_incontinence",
                         "milestone_h_and_y",
                         "milestone_s_and_e",
                        ]
milestone_cols.extend(additional_milestones)
merged_df['any_milestone'] = merged_df[milestone_cols].max(axis=1)
milestone_cols.append('any_milestone')

In [ ]:
df_all_items_indexed = df_all_items.set_index('item')
all_methods = df_all_items_indexed.columns

all_items_results = {}
for method in all_methods:
    all_items_results[method + " (all)"] = merged_df[df_all_items_indexed.index].dot(df_all_items_indexed[method])

merged_df = pd.concat([merged_df, pd.DataFrame(all_items_results)], axis=1)

df_self_report_indexed = df_self_report.set_index('item')
self_reported_methods = [method for method in all_methods if method not in ['MDS-UPDRS Part 3', 'MoCA']]

self_report_results = {}
for method in self_reported_methods:
    self_report_results[method + " (self_reported)"] = merged_df[df_self_report_indexed.index].dot(df_self_report_indexed[method])

merged_df = pd.concat([merged_df, pd.DataFrame(self_report_results)], axis=1)

methods = [c for c in merged_df.columns if c.endswith('(all)') or c.endswith('(self_reported)')]

In [ ]:
def calculate_time_to_milestone(grp):
    """
    grp is the subset of df for a single PATNO, sorted by visit_month.
    We'll return that group with a new 'time_to_milestone' column
    using the rule:
      - 0 if this row's any_milestone == 1
      - (earliest_milestone_month - visit_month) if the row is before the first milestone
      - NaN if the row is after the first milestone
      - NaN if the patient never reaches a milestone
    """
    milestone_months = grp.loc[grp["any_milestone"] == 1, "visit_month"]
    
    if milestone_months.empty:
        # No milestone at all -> all NaN
        return grp.assign(time_to_milestone=float("nan"))

    first_milestone_month = milestone_months.min()
    
    # Vectorized computation
    time_to_milestone = (
        (first_milestone_month - grp["visit_month"])
        .where(grp["visit_month"] < first_milestone_month, float("nan"))
    )
    time_to_milestone.loc[grp["any_milestone"] == 1] = 0  # Milestone visits are zero

    return grp.assign(time_to_milestone=time_to_milestone)

# Apply function patient-by-patient
merged_df = (
    merged_df
    .sort_values(["PATNO", "visit_month"])
    .groupby("PATNO", group_keys=False)
    .apply(calculate_time_to_milestone)
)

In [ ]:
df_valid = merged_df.dropna(subset=["time_to_milestone"]).copy()

all_items_methods = [method + ' (all)' for method in all_methods]
rs = []

for method in methods:
    df_method = df_valid.dropna(subset=[method])
    r, p_value = pearsonr(df_method["time_to_milestone"], df_method[method])
    if method in all_items_methods:
        rs.append(r)
    print(f'{method}: rho={r}, p={p_value}')

In [ ]:
# Compare correlations of methods using all items

ax = sns.barplot(y=all_methods, x=rs, palette='Wistia')
ax.vlines(x = min(rs[:5]), ymin = -.5, ymax = 10.5, linestyle='dashed', color='red')
ax.set_xlabel('Correlation coefficient')
plt.title("Correlation coefficients between progression index and time to milestone", x=0.38)
plt.show()